# **Reading the Data from Database**

In [1]:
import sqlite3
import pandas as pd

## **Step 1 - Reading the Tables from Database file**

In [2]:
# Read the code below and write your observation in the next cell

conn = sqlite3.connect('data/eng_subtitles_database.db')
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
print(cursor.fetchall())

[('zipfiles',)]


**In the above cell, I am able to read the table inside the database. As mentioned earlier, table name is `zipfiles`. We also know from README.txt that this table contains three columns: 'num', 'name' and 'content'.**

## **Step 2 - Reading the columns of Table**

In [3]:
cursor.execute("PRAGMA table_info('zipfiles')")
cols = cursor.fetchall()
for col in cols:
    print(col[1])

num
name
content


**The above code helps in checking the column names in the database table.**

**Let's now use `SELECT * FROM zipfiles` to read all the data into a `df` variable.**

## **Step 3 - Loading the Database Table inside a Pandas DataFrame**

In [4]:
df = pd.read_sql_query("""SELECT * FROM zipfiles""", conn)
df.head()

,num,name,content
0,9180533,the.message.(1976).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x1c\xa9\x...
1,9180583,here.comes.the.grump.s01.e09.joltin.jack.in.bo...,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x17\xb9\x...
2,9180592,yumis.cells.s02.e13.episode.2.13.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00L\xb9\x99V...
3,9180594,yumis.cells.s02.e14.episode.2.14.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00U\xa9\x99V...
4,9180600,broker.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x001\xa9\x99V...


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82498 entries, 0 to 82497
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   num      82498 non-null  int64 
 1   name     82498 non-null  object
 2   content  82498 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.9+ MB


**Looks like the `content` column donot contain the subtitles text. Instead as mentioned in README.txt, it might be latin-1 encoded.**

## **Step 4 - Printing `content` of 0th Row**

In [6]:
b_data = df.iloc[0, 2]

# here 2 represent the index of content column
# 0 represents the row number

In [7]:
# print(b_data)

**From the content, it appears to start with the bytes "PK\x03\......", which suggests that it might be a ZIP archive file. How do I know it? Experience! I have worked with something similar earlier.**

## **Step 5 - Unzipping the content of 385th row and decoding using `latin-1`**

In [8]:
import zipfile
import io

# Assuming 'content' is the binary data from your database
binary_data = df.iloc[385, 2]

# Decompress the binary data using the zipfile module
with io.BytesIO(binary_data) as f:
    with zipfile.ZipFile(f, 'r') as zip_file:
        # Reading only one file in the ZIP archive
        subtitle_content = zip_file.read(zip_file.namelist()[0])

# Now 'subtitle_content' should contain the extracted subtitle content
print(subtitle_content.decode('latin-1')[:320])  # Assuming the content is latin-1 encoded text

1
00:00:06,000 --> 00:00:12,074
Watch any video online with Open-SUBTITLES
Free Browser extension: osdb.link/ext

2
00:00:15,370 --> 00:00:16,506
You lose everything, my girl.

3
00:00:16,530 --> 00:00:19,360
So you've said - four times.

4
00:00:20,330 --> 00:00:22,120
I definitely had
it on yesterday.


**Look's like it worked.**

## **Step 6 - Applying the above Function on the Entire Data**

In [9]:
import zipfile
import io

count = 0

def decode_method(binary_data):
    global count
    # Decompress the binary data using the zipfile module
    # print(count, end=" ")
    count += 1
    with io.BytesIO(binary_data) as f:
        with zipfile.ZipFile(f, 'r') as zip_file:
            # Assuming there's only one file in the ZIP archive
            subtitle_content = zip_file.read(zip_file.namelist()[0])
    
    # Now 'subtitle_content' should contain the extracted subtitle content
    return subtitle_content.decode('latin-1')  # Assuming the content is UTF-8 encoded text

In [10]:
df['file_content'] = df['content'].apply(decode_method)

df.head()

,num,name,content,file_content
0,9180533,the.message.(1976).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x1c\xa9\x...,"1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch an..."
1,9180583,here.comes.the.grump.s01.e09.joltin.jack.in.bo...,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x17\xb9\x...,"1\r\n00:00:29,359 --> 00:00:32,048\r\nAh! Ther..."
2,9180592,yumis.cells.s02.e13.episode.2.13.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00L\xb9\x99V...,"1\r\n00:00:53,200 --> 00:00:56,030\r\n<i>Yumi'..."
3,9180594,yumis.cells.s02.e14.episode.2.14.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00U\xa9\x99V...,"1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch an..."
4,9180600,broker.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x001\xa9\x99V...,"ï»¿1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch..."


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82498 entries, 0 to 82497
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   num           82498 non-null  int64 
 1   name          82498 non-null  object
 2   content       82498 non-null  object
 3   file_content  82498 non-null  object
dtypes: int64(1), object(3)
memory usage: 2.5+ MB


In [12]:
df.columns = ['num', 'name', 'content', 'subtitles']

df.tail()

,num,name,content,subtitles
82493,9521935,the.prophets.game.(2000).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\xb8\xa6\x...,"ï»¿1\r\n00:01:16,284 --> 00:01:19,537\r\nGod,\..."
82494,9521937,west.beirut.(1998).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x13\x97\x...,"1\r\n00:00:06,000 --> 00:00:12,074\r\napi.Open..."
82495,9521938,frankenstein.the.true.story.(1973).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00$\x97\x9aV...,"1\r\n00:00:01,001 --> 00:00:04,630\r\n(Dramati..."
82496,9521940,frankenstein.the.true.story.(1973).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x00\x97\x...,"1\r\n00:00:06,000 --> 00:00:12,074\r\nAdvertis..."
82497,9521941,zombie.island.massacre.(1984).eng.1cd,"b'PK\x03\x04\x14\x00\x00\x00\x08\x00,\x97\x9aV...","1\r\n00:00:01,919 --> 00:00:03,253\r\n(Sharp w..."


In [13]:
df_new = df[['num', 'name', 'subtitles']]

df_new.head()

,num,name,subtitles
0,9180533,the.message.(1976).eng.1cd,"1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch an..."
1,9180583,here.comes.the.grump.s01.e09.joltin.jack.in.bo...,"1\r\n00:00:29,359 --> 00:00:32,048\r\nAh! Ther..."
2,9180592,yumis.cells.s02.e13.episode.2.13.(2022).eng.1cd,"1\r\n00:00:53,200 --> 00:00:56,030\r\n<i>Yumi'..."
3,9180594,yumis.cells.s02.e14.episode.2.14.(2022).eng.1cd,"1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch an..."
4,9180600,broker.(2022).eng.1cd,"ï»¿1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch..."


In [14]:
# df_new.to_csv('data/subtitles_extracted_data.csv', index=False)

In [15]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

def clean(doc):
    regex = '[^a-zA-Z.]'
    doc = re.sub(regex, ' ', doc)

    doc = doc.lower()

    tokens = nltk.word_tokenize(doc)

    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]

    return ' '.join(lemmatized_tokens)

In [16]:
from tqdm import tqdm

tqdm.pandas()

df['subtitles'] = df['subtitles'].progress_apply(lambda x: clean(x))

df.head()

100%|██████████| 82498/82498 [37:41<00:00, 36.49it/s]    


,num,name,content,subtitles
0,9180533,the.message.(1976).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x1c\xa9\x...,watch any video online with open subtitle free...
1,9180583,here.comes.the.grump.s01.e09.joltin.jack.in.bo...,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x17\xb9\x...,ah there s princess dawn and terry with the bl...
2,9180592,yumis.cells.s02.e13.episode.2.13.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00L\xb9\x99V...,i yumi s cell i i episode extremely polite yum...
3,9180594,yumis.cells.s02.e14.episode.2.14.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00U\xa9\x99V...,watch any video online with open subtitle free...
4,9180600,broker.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x001\xa9\x99V...,watch any video online with open subtitle free...


In [17]:
print(df['subtitles'][0][0:1000])

watch any video online with open subtitle free browser extension osdb.link ext in the name of god the most gracious the most merciful . from muhammad the messenger of god to heraclius the emperor of byzantium . greeting to him who is the follower of righteous guidance . i bid you to hear the divine call . i am the messenger of god to the people accept islam for your salvation . he speaks of a new prophet in arabia . wa it like this when john the baptist came to king herod out of the desert cry about salvation to muqawqis patriarch of alexandria . kisra emperor of persia . muhammad call you with the call of god . accept islam for your salvation ... embrace islam . you come out of the desert smelling of camel and goat . to tell persia where he should kneel muhammad messenger of god . who gave him this authority god sent muhammad a a mercy to mankind . the scholar and historian of islam the university of al azhar in cairo the high islamic congress of the shiat in lebanon the maker of this

In [18]:
import re

def extract_metadata(name):
    # First check if the name contains season and episode information (series)
    series_match = re.match(r"([a-zA-Z\s\.]+)\.s\d{2}\.e\d{2}", name)
    
    if series_match:
        # For series: Extract name and replace dots with spaces
        extracted_name = series_match.group(1).strip().replace('.', ' ')
        
        # Extract season, episode, and year
        season_match = re.search(r"\.s(\d+)", name)
        season = season_match.group(1) if season_match else None
        
        episode_match = re.search(r"\.e(\d+)", name)
        episode = episode_match.group(1) if episode_match else None
        
        year_match = re.search(r"\((\d{4})\)", name)
        year = year_match.group(1) if year_match else None
        
    else:
        # For movies: Extract name before the year (if no season and episode)
        extracted_name = re.match(r"([a-zA-Z\s\.]+)\.\(\d{4}\)", name).group(1).strip().replace('.', ' ') if re.match(r"([a-zA-Z\s\.]+)\.\(\d{4}\)", name) else name
        
        # Movie name doesn't have season or episode, so only extract the year
        season, episode = None, None
        
        year_match = re.search(r"\((\d{4})\)", name)
        year = year_match.group(1) if year_match else None
    
    return extracted_name, season, episode, year

In [19]:
# Apply the function to the 'name' column

df[['extracted_name', 'season', 'episode', 'year']] = df['name'].apply(lambda x: pd.Series(extract_metadata(x)))

In [20]:
df = df[['num', 'extracted_name', 'season', 'episode', 'year', 'subtitles']]

df.head(3)

,num,extracted_name,season,episode,year,subtitles
0,9180533,the message,None,None,1976,watch any video online with open subtitle free...
1,9180583,here comes the grump,01,09,1969,ah there s princess dawn and terry with the bl...
2,9180592,yumis cells,02,13,2022,i yumi s cell i i episode extremely polite yum...


In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82498 entries, 0 to 82497
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   num             82498 non-null  int64 
 1   extracted_name  82498 non-null  object
 2   season          61829 non-null  object
 3   episode         61829 non-null  object
 4   year            80446 non-null  object
 5   subtitles       82498 non-null  object
dtypes: int64(1), object(5)
memory usage: 3.8+ MB


In [22]:
df['season'] = df['season'].fillna('movie')
df['episode'] = df['episode'].fillna('movie')
df['year'] = df['year'].fillna('unknown')

df.rename(columns={'extracted_name': 'name'}, inplace=True)
df.rename(columns={'num': 'movie_id'}, inplace=True)

In [23]:
df.head()

,movie_id,name,season,episode,year,subtitles
0,9180533,the message,movie,movie,1976,watch any video online with open subtitle free...
1,9180583,here comes the grump,01,09,1969,ah there s princess dawn and terry with the bl...
2,9180592,yumis cells,02,13,2022,i yumi s cell i i episode extremely polite yum...
3,9180594,yumis cells,02,14,2022,watch any video online with open subtitle free...
4,9180600,broker,movie,movie,2022,watch any video online with open subtitle free...


In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82498 entries, 0 to 82497
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   movie_id   82498 non-null  int64 
 1   name       82498 non-null  object
 2   season     82498 non-null  object
 3   episode    82498 non-null  object
 4   year       82498 non-null  object
 5   subtitles  82498 non-null  object
dtypes: int64(1), object(5)
memory usage: 3.8+ MB


In [26]:
# df.to_csv('data/subtitles_metadata.csv', index=False)